In [6]:
import pandas as pd
# Load datasets
print("Loading AIDev datasets...")

# AI Agent PRs
pr_df = pd.read_parquet("hf://datasets/hao-li/AIDev/pull_request.parquet")
pr_task_type_df = pd.read_parquet("hf://datasets/hao-li/AIDev/pr_task_type.parquet")
ai_perf_prs = (
    pr_df
    .merge(
        pr_task_type_df[["id", "type", "reason"]],
        on="id",
        how="inner"
    )
    .query("type == 'perf'")
    .copy()
)
ai_perf_prs['classification_reason'] = ai_perf_prs['reason']
ai_perf_prs['author_type'] = 'AI Agent'

# Human PRs
human_pr_df = pd.read_parquet("hf://datasets/hao-li/AIDev/human_pull_request.parquet")
human_pr_task_type_df = pd.read_parquet("hf://datasets/hao-li/AIDev/human_pr_task_type.parquet")
human_perf_prs = (
    human_pr_df
    .merge(
        human_pr_task_type_df[["id", "type", "reason"]],
        on="id",
        how="inner"
    )
    .query("type == 'perf'")
    .copy()
)
human_perf_prs['classification_reason'] = human_perf_prs['reason']
human_perf_prs['author_type'] = 'Human'
human_perf_prs['agent'] = 'Human'

# Store original counts
original_ai_count = len(ai_perf_prs)
original_human_count = len(human_perf_prs)

# Repository data for language info
all_repo_df = pd.read_parquet("hf://datasets/hao-li/AIDev/all_repository.parquet")

# Get list of performance PR IDs we care about
perf_pr_ids = set(ai_perf_prs['id'].tolist() + human_perf_prs['id'].tolist())
print(f"\n✓ Performance PR IDs to process: {len(perf_pr_ids):,}")

# PR commits details - FILTER FIRST, then aggregate
print("\nProcessing commit details (filtering to performance PRs only)...")
pr_commits_details = pd.read_parquet("hf://datasets/hao-li/AIDev/pr_commit_details.parquet")

# Pr commit details for human PRs
human_pr_commit_details = pd.read_parquet("./datasets/human_pr/human_pr_commit_details.parquet")


pr_commits_details = pd.concat(
    [pr_commits_details, human_pr_commit_details],
    ignore_index=True
)

if 'pr_id' in pr_commits_details.columns:
    print(f"  Total commit records in dataset: {len(pr_commits_details):,}")
    
    # FILTER: Keep only commits for performance PRs
    pr_commits_filtered = pr_commits_details[pr_commits_details['pr_id'].isin(perf_pr_ids)].copy()
    print(f"  Filtered to performance PRs: {len(pr_commits_filtered):,} commit records")
    print(f"  Unique performance PRs with commits: {pr_commits_filtered['pr_id'].nunique():,}")
    
    # ADDITIONAL FILTERING: Remove commits with null filename
    if 'filename' in pr_commits_filtered.columns:
        before_filename_filter = len(pr_commits_filtered)
        pr_commits_filtered = pr_commits_filtered[pr_commits_filtered['filename'].notna()].copy()
        print(f"  Filtered out null filenames: {before_filename_filter - len(pr_commits_filtered):,} records removed")
        print(f"  Remaining after filename filter: {len(pr_commits_filtered):,} commit records")
        
    # ADDITIONAL FILTERING: Remove config/metadata-only files
    if 'filename' in pr_commits_filtered.columns:
        before_config_filter = len(pr_commits_filtered)
        
        # Define patterns for non-code files to exclude
        config_patterns = [
            r'^\.mvn/',                          # Maven wrapper configs
            r'^\.gradle/',                       # Gradle configs
            r'^\.idea/',                         # IntelliJ configs
            r'^\.vscode/',                       # VSCode configs
            r'^\.github/workflows/',             # GitHub Actions (unless it's code)
            r'\.properties$',                    # Properties files
            r'\.xml$',                           # XML config files (pom.xml, etc.)
            r'\.yml$',                           # YAML configs
            r'\.yaml$',                          # YAML configs
            r'\.json$',                          # JSON configs (package.json, etc.)
            r'\.md$',                            # Markdown docs
            r'\.txt$',                           # Text files
            r'\.gitignore$',                     # Git configs
            r'\.dockerignore$',                  # Docker ignore files
            r'/Dockerfile$',                     # Dockerfiles (anywhere in path)
            r'^Dockerfile$',                     # Dockerfile at root
            r'/docker-compose',                  # Docker compose (anywhere)
            r'^docker-compose',                  # Docker compose at root
            r'\.lock$',                          # Lock files (package-lock, yarn.lock)
            r'^LICENSE',                         # License files
            r'^README',                          # README files
        ]
        
        config_pattern = '|'.join(config_patterns)
        
        # Mark config files
        pr_commits_filtered['is_config_file'] = pr_commits_filtered['filename'].str.contains(
            config_pattern, case=False, na=False, regex=True
        )
        
        # Keep track of which files are code files per PR
        pr_commits_filtered['is_code_file'] = ~pr_commits_filtered['is_config_file']
        
        # For each PR, check if it has ANY code files
        pr_has_code = pr_commits_filtered.groupby('pr_id')['is_code_file'].any().reset_index()
        pr_has_code.columns = ['pr_id', 'has_code_files']
        
        # Filter to keep only PRs that have at least one code file
        pr_commits_filtered = pr_commits_filtered.merge(pr_has_code, on='pr_id', how='left')
        pr_commits_filtered = pr_commits_filtered[pr_commits_filtered['has_code_files']].copy()
        
        # Clean up temporary columns
        pr_commits_filtered = pr_commits_filtered.drop(columns=['is_config_file', 'is_code_file', 'has_code_files'])
        
        print(f"  Filtered out config-only commits: {before_config_filter - len(pr_commits_filtered):,} records removed")
        print(f"  Remaining after config filter: {len(pr_commits_filtered):,} commit records")
    
    # ADDITIONAL FILTERING: Remove merge commits
    if 'message' in pr_commits_filtered.columns:
        before_merge_filter = len(pr_commits_filtered)
        # Common merge commit patterns
        merge_patterns = [
            r'^Merge\s+branch',
            r'^Merge\s+pull\s+request',
            r'^Merge\s+remote-tracking\s+branch',
            r'^Merge\s+.*\s+into\s+',
            r"^Merged\s+in\s+",
        ]
        merge_pattern = '|'.join(merge_patterns)
        pr_commits_filtered = pr_commits_filtered[
            ~pr_commits_filtered['message'].str.match(merge_pattern, case=False, na=False)
        ].copy()
        print(f"  Filtered out merge commits: {before_merge_filter - len(pr_commits_filtered):,} records removed")
        print(f"  Remaining after merge filter: {len(pr_commits_filtered):,} commit records")

        # FINAL FILTER: drop PRs whose repositories were deleted
        deleted_repo_pr_ids = {3271610326, 3209206554}
        before_deleted_repo_filter = len(pr_commits_filtered)
        pr_commits_filtered = pr_commits_filtered[~pr_commits_filtered['pr_id'].isin(deleted_repo_pr_ids)].copy()
        removed_deleted_repo_commits = before_deleted_repo_filter - len(pr_commits_filtered)
        if removed_deleted_repo_commits > 0:
            print(f"  Removed deleted repo PR commits: {removed_deleted_repo_commits:,} records removed")
        print(f"  Remaining after deleted repo filter: {len(pr_commits_filtered):,} commit records")
    
    print(f"  Unique performance PRs after all filters: {pr_commits_filtered['pr_id'].nunique():,}")
    
    if len(pr_commits_filtered) > 0:
        # AGGREGATE: Now aggregate only the filtered commits
        commit_aggregated = pr_commits_filtered.groupby('pr_id').agg({
            'additions': 'sum',      # Total lines added across all commits
            'deletions': 'sum',      # Total lines deleted across all commits
            'patch': lambda x: '\n\n'.join([str(p) for p in x if pd.notna(p)])  # Concatenate all patches
        }).reset_index()
        
        # Add derived metrics
        commit_aggregated['num_commits'] = pr_commits_filtered.groupby('pr_id').size().values
        
        # Calculate patch length (for analysis)
        commit_aggregated['patch_length'] = commit_aggregated['patch'].str.len()
        
        print(f"  ✓ Aggregated to {len(commit_aggregated):,} unique performance PRs")
        print(f"  Avg commits per PR: {commit_aggregated['num_commits'].mean():.1f}")
        
        # Merge commit stats into AI Agent PR table
        ai_perf_prs = ai_perf_prs.merge(
            commit_aggregated,
            left_on='id',
            right_on='pr_id',
            how='left'
        )
        if 'pr_id' in ai_perf_prs.columns:
            ai_perf_prs = ai_perf_prs.drop(columns=['pr_id'])
        
        # Filter to keep only PRs with commit data
        ai_before_filter = len(ai_perf_prs)
        ai_with_commits = ai_perf_prs[ai_perf_prs['additions'].notna()].copy()
        print(f"  AI Agent PRs with commit data: {len(ai_with_commits):,} / {ai_before_filter:,} ({len(ai_with_commits)/ai_before_filter*100:.1f}%)")
        
        # Merge commit stats into Human PR table
        human_perf_prs = human_perf_prs.merge(
            commit_aggregated,
            left_on='id',
            right_on='pr_id',
            how='left'
        )
        if 'pr_id' in human_perf_prs.columns:
            human_perf_prs = human_perf_prs.drop(columns=['pr_id'])
        
        # Filter to keep only PRs with commit data
        human_before_filter = len(human_perf_prs)
        human_with_commits = human_perf_prs[human_perf_prs['additions'].notna()].copy()
        print(f"  Human PRs with commit data: {len(human_with_commits):,} / {human_before_filter:,} ({len(human_with_commits)/human_before_filter*100:.1f}%)")
    else:
        print("  ⚠ No commits found for performance PRs after filtering")
        # Create empty dataframes with same structure
        ai_with_commits = ai_perf_prs.iloc[0:0].copy()
        human_with_commits = human_perf_prs.iloc[0:0].copy()
    
else:
    print('⚠ pr_commit_details missing pr_id column; skipping commit merges.')
    # Create empty dataframes
    ai_with_commits = ai_perf_prs.iloc[0:0].copy()
    human_with_commits = human_perf_prs.iloc[0:0].copy()

print(f"\n{'='*80}")
print(f"SUMMARY")
print(f"{'='*80}")
print(f"Original Performance PRs:")
print(f"  AI Agent: {original_ai_count:,}")
print(f"  Human: {original_human_count:,}")
print(f"  Total: {original_ai_count + original_human_count:,}")
print(f"\nAfter Commit Filtering:")
print(f"✓ AI Agent Performance PRs: {len(ai_with_commits):,}")
print(f"✓ Human Performance PRs: {len(human_with_commits):,}")
print(f"✓ Total Performance PRs: {len(ai_with_commits) + len(human_with_commits):,}")

# Distribution by AI agent
if len(ai_with_commits) > 0:
    print(f"\nAI Agent Distribution:")
    for agent, count in ai_with_commits['agent'].value_counts().items():
        pct = count / len(ai_with_commits) * 100
        print(f"  {agent:20s} {count:5,d} ({pct:5.1f}%)")

# Commit statistics summary
if len(ai_with_commits) > 0 or len(human_with_commits) > 0:
    print(f"\n{'='*80}")
    print(f"COMMIT STATISTICS")
    print(f"{'='*80}")
    
    for author_type, df in [('AI Agent', ai_with_commits), ('Human', human_with_commits)]:
        if len(df) > 0:
            print(f"\n{author_type}:")
            print(f"  PRs with commit data: {len(df):,}")
            print(f"  Avg commits per PR: {df['num_commits'].mean():.1f}")
            print(f"  Median commits per PR: {df['num_commits'].median():.1f}")
            print(f"  Avg additions: {df['additions'].mean():.0f} lines")
            print(f"  Median additions: {df['additions'].median():.0f} lines")
            print(f"  Avg deletions: {df['deletions'].mean():.0f} lines")
            print(f"  Median deletions: {df['deletions'].median():.0f} lines")

# Save valid PR IDs for reuse
valid_ids_output_path = 'valid_perf_pr_ids.csv'
valid_pr_ids_series = pd.concat([
    ai_with_commits['id'],
    human_with_commits['id'],
], ignore_index=True).dropna().drop_duplicates()
valid_pr_ids_df = (
    valid_pr_ids_series.to_frame(name='id')
    .sort_values('id')
    .reset_index(drop=True)
)
valid_pr_ids_df.to_csv(valid_ids_output_path, index=False)
print(f"\nSaved {len(valid_pr_ids_df):,} valid performance PR IDs to {valid_ids_output_path}")

print(f"\n{'='*80}")


Loading AIDev datasets...

✓ Performance PR IDs to process: 428

Processing commit details (filtering to performance PRs only)...
  Total commit records in dataset: 712,528
  Filtered to performance PRs: 8,015 commit records
  Unique performance PRs with commits: 423
  Filtered out null filenames: 46 records removed
  Remaining after filename filter: 7,969 commit records
  Filtered out config-only commits: 39 records removed
  Remaining after config filter: 7,930 commit records
  Filtered out merge commits: 2,945 records removed
  Remaining after merge filter: 4,985 commit records
  Removed deleted repo PR commits: 74 records removed
  Remaining after deleted repo filter: 4,911 commit records
  Unique performance PRs after all filters: 407
  ✓ Aggregated to 407 unique performance PRs
  Avg commits per PR: 12.1
  AI Agent PRs with commit data: 324 / 340 (95.3%)
  Human PRs with commit data: 83 / 88 (94.3%)

SUMMARY
Original Performance PRs:
  AI Agent: 340
  Human: 88
  Total: 428

Afte